In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
cd My Drive/palpx/CNN_scratch

In [ ]:
ls

In [ ]:
#FETCHING THE DATA HERE USIGN WGET COMMAND..
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/3362/31148/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1597255669&Signature=CbeNhcxBBqdERi3eIktwufCPIgS9V0Dx%2F26QrKUaOxxa40aZ4pGitoYU1feFnclh8S8NUzwAIgXzDij9YZJE%2BNU0FXCWSEVoz2CS%2BC3PrEesYMzkV%2F%2B18%2BzpdGB7d4Vp5GRTGvxTWKvLLneRGQWnCewSOIc5ITmZAML4GC2coT2vJ2f3hQYumWDKOjieVz7L%2BhQbcUZLvry4mfnnFvCq7HJnSYEMwjW1HVduOu4CKC2uYMUraG1%2Fkxy8wc%2FOYYlyWRU3zIPn1MTbG2%2BUq2O4pAKctL%2BqJR%2FWd6BO70st365ilaUo88aTaS6UuwSKaH6okv3Kg3gfhCEoHJfbuFIkZw%3D%3D&response-content-disposition=attachment%3B+filename%3Ddogs-vs-cats.zip" -c -O 'dogs-vs-cats.zip'

In [ ]:
!sudo apt-get install unzip

In [ ]:
!unzip train.zip 

In [ ]:
!unzip test1.zip 

In [1]:
import os
from sklearn.model_selection import train_test_split
import cv2 as cv2
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [2]:
def _shuffle(X, Y):
    m = Y.shape[0]
    indices = np.arange(m)
    np.random.shuffle(indices)

    return X[indices], Y[indices]

def batches_generator(X_train, Y_train=None, batch_size=256, shuffle=False):
    m = X_train.shape[0]
    
    if Y_train is None:
        for j in range(0, m, batch_size):
            yield X_train[j:j+batch_size]
    
    else:
        if shuffle:
            X_train, Y_train = _shuffle(X_train, Y_train)

        for j in range(0, m, batch_size):
            yield X_train[j:j+batch_size], Y_train[j:j+batch_size]

In [3]:
import numpy as np
from abc import ABCMeta, abstractmethod

class Layer(object):
    __metaclass__ = ABCMeta

    @abstractmethod
    def init_params(self):
        pass

    @abstractmethod
    def set_input_shape(self, in_shape):
        pass

    @abstractmethod
    def get_out_shape(self):
        pass

    @abstractmethod
    def forward(self, X):
        pass

    @abstractmethod
    def backward(self, dZ):
        pass

    @abstractmethod
    def update_params(self, lr):
        pass

    @abstractmethod
    def get_nparams(self):
        pass

class Input(object):

    LAYER_TYPE = 'input'

    def __init__(self, in_shape):
        self.in_shape = tuple(in_shape)
        self.out_shape = tuple(in_shape)
        self.nparams = 0

class Conv2D(Layer):

    LAYER_TYPE = 'conv'

    def __init__(self,
                 kernels,
                 k_size=3,
                 strides=2,
                 padding=1):
        self.kernels = kernels
        self.k_size = k_size
        self.strides = strides
        self.padding = padding

    def _zero_pad(self, X, pad):
        return np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)),
                      'constant', constant_values=(0, 0))

    def _conv_single_step(self, X_slice, W, b):
        return np.sum(X_slice * W) + float(b)

    def set_input_shape(self, in_shape):
        self.in_shape = in_shape

    def get_out_shape(self):
        (n_H_prev, n_W_prev, n_C_prev) = self.in_shape
        n_H = int((n_H_prev - self.k_size + 2 * self.padding) / self.strides) + 1
        n_W = int((n_W_prev - self.k_size + 2 * self.padding) / self.strides) + 1
        return (n_H, n_W, self.kernels)

    def init_params(self):
        self.W = np.random.randn(self.k_size,
                                 self.k_size,
                                 self.in_shape[2],
                                 self.kernels)
        self.b = np.random.randn(1, 1, 1, self.kernels)

    def forward(self, X):
        self.X = X
        (m, n_H_prev, n_W_prev, n_C_prev) = X.shape
        (f, f, n_C_prev, n_C) = self.W.shape

        n_H = int((n_H_prev - f + 2 * self.padding) / self.strides) + 1
        n_W = int((n_W_prev - f + 2 * self.padding) / self.strides) + 1

        Z = np.zeros((m, n_H, n_W, n_C))
        X_pad = self._zero_pad(self.X, self.padding)

        for i in range(m):
            x_pad = X_pad[i]
            for h in range(n_H):
                for w in range(n_W):
                    for c in range(n_C):
                        vert_start = h + h * (self.strides - 1)
                        vert_end = vert_start + f
                        horiz_start = w + w * (self.strides - 1)
                        horiz_end = horiz_start + f

                        x_slice = x_pad[vert_start:vert_end, horiz_start:horiz_end, :]
                        Z[i, h, w, c] = self._conv_single_step(x_slice,
                                                               self.W[:, :, :, c],
                                                               self.b[:, :, :, c])
        return Z

    def backward(self, dZ):
        (m, n_H_prev, n_W_prev, n_C_prev) = self.X.shape
        (f, f, n_C_prev, n_C) = self.W.shape
        (m, n_H, n_W, n_C) = dZ.shape

        self.dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))
        self.dW = np.zeros((f, f, n_C_prev, n_C))
        self.db = np.zeros((1, 1, 1, n_C))

        X_pad = self._zero_pad(self.X, self.padding)
        dA_prev_pad = self._zero_pad(self.dA_prev, self.padding)

        for i in range(m):
            x_pad = X_pad[i]
            da_prev_pad = dA_prev_pad[i]
            
            for h in range(n_H):
                for w in range(n_W):
                    for c in range(n_C):
                        vert_start = h + h * (self.strides - 1)
                        vert_end = vert_start + f
                        horiz_start = w + w * (self.strides - 1)
                        horiz_end = horiz_start + f

                        x_slice = x_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                        da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += self.W[:, :, :, c] * dZ[i, h, w, c]
                        self.dW[:,:,:,c] += x_slice * dZ[i, h, w, c]
                        self.db[:,:,:,c] += dZ[i, h, w, c]
                    
            self.dA_prev[i, :, :, :] = da_prev_pad[self.padding:-self.padding, self.padding:-self.padding, :]
        return self.dA_prev

    def update_params(self, lr):
        self.W  = self.W - lr * self.dW
        self.b  = self.b - lr * self.db

    def get_nparams(self):
        return np.product(self.W.shape) + np.product(self.b.shape)

class Flatten(Layer):

    LAYER_TYPE = 'flatten'

    def init_params(self):
        pass

    def set_input_shape(self, in_shape):
        self.in_shape = in_shape

    def get_out_shape(self):
        return np.product(self.in_shape)

    def forward(self, X):
        self.X = X
        return np.reshape(X, (-1, self.get_out_shape()))

    def backward(self, dZ):
        return np.reshape(np.array(dZ), [-1] + list(self.in_shape))

    def update_params(self, lr):
        pass

    def get_nparams(self):
        return 0

class Dense(Layer):

    LAYER_TYPE = 'dense'

    def __init__(self, units):
        self.units = units

    def set_input_shape(self, in_shape):
        self.in_shape = in_shape

    def get_out_shape(self):
        return self.units

    def init_params(self):
        self.W = np.random.randn(self.in_shape, self.units)
        self.b = np.zeros((1, self.units))

    def forward(self, X):
        self.X = X
        return np.matmul(X, self.W) + self.b

    def backward(self, dZ):
        m = self.X.shape[0]
        self.dW = np.dot(self.X.T, dZ) / m
        self.db = np.sum(dZ, axis=0) / m
        return np.dot(dZ, self.W.T)

    def update_params(self, lr):
        self.W = self.W - lr * self.dW
        self.b = self.b - lr * self.db

    def get_nparams(self):
        return np.product(self.W.shape) + np.product(self.b.shape)
    
    ################################################################
class Max_pool(Layer):

    LAYER_TYPE = 'pool'

    def __init__(self, units):
        self.units = units

    def set_input_shape(self, in_shape):
        self.in_shape = in_shape

    def get_out_shape(self):
        return self.units

    def init_params(self):
        self.W = np.random.randn(self.in_shape, self.units)
        self.b = np.zeros((1, self.units))
    def forward(self,X):
        X_col = im2col_indices(X_reshaped, size, size, padding=0, stride=stride)
        # Next, at each possible patch location, i.e. at each column, we're taking the max index
        max_idx = np.argmax(X_col, axis=0)
        # Finally, we get all the max value at each column
        out = X_col[max_idx, range(max_idx.size)]
        # Reshape to the output size:
        out = out.reshape(h_out, w_out, n, d)
        out = out.transpose(2, 3, 0, 1)
        return out
    def backward(self,dZ):
        dX_col = np.zeros_like(X_col)
        dout_flat = dout.transpose(2, 3, 0, 1).ravel()
        dX_col[max_idx, range(max_idx.size)] = dout_flat
        dX = col2im_indices(dX_col, (n * d, 1, h, w), size, size, padding=0, stride=stride)
        dX = dX.reshape(X.shape)
        return dX
        

    def update_params(self, lr):
        self.W = self.W - lr * self.dW
        self.b = self.b - lr * self.db

    def get_nparams(self):
        return np.product(self.W.shape) + np.product(self.b.shape)

class Activation(Layer):

    LAYER_TYPE = 'act'

    ACTIVATIONS = ['relu', 'softmax','sigmoid']
    def __init__(self, act='relu'):
        assert act in Activation.ACTIVATIONS
        self.act = act

    def init_params(self):
        pass

    def set_input_shape(self, in_shape):
        self.in_shape = in_shape

    def get_out_shape(self):
        return self.in_shape

    def _softmax(self):
        exp_vals = np.exp(self.X - self.X.max(axis=1, keepdims=True))
        return exp_vals/ np.sum(exp_vals, axis=1, keepdims=True)

    def _relu(self):
        return np.maximum(self.X, 0)
    def _sigmoid(self):
        return 1/(1 + np.exp(-self.X)) 

    def forward(self, X):
        self.X = X
        if self.act == Activation.ACTIVATIONS[0]:
            return self._relu()
        if self.act == Activation.ACTIVATIONS[1]:
            return self._softmax()
        if self.act == Activation.ACTIVATIONS[2]:
            return self._sigmoid()

    def backward(self, dZ):
        if self.act == Activation.ACTIVATIONS[0]:
            return dZ * (self.X >= 1)

        if self.act == Activation.ACTIVATIONS[1]:
            m = dZ.shape[0]
            grad = self._softmax()
            grad[range(m), np.argmax(dZ, axis=1)] -= 1
            return grad / m
        if self.act == Activation.ACTIVATIONS[2]:
            m = dZ.shape[0]
            grad = self._sigmoid()
            return dZ * grad * (1 - grad)

    def update_params(self, lr):
        pass

    def get_nparams(self):
        return 0


In [15]:

from tqdm import tqdm

class Model(object):

    LOSS_TO_METHOD = {
        'crossentropy': crossentropy_loss
    }

    def __init__(self):
        self.compiled = False
        self.layers = []

    def add_layers(self, layer):
        self.layers.append(layer)

    def compile(self, loss='crossentropy', lr=0.3):
#         print(n_epochs)
        
        assert loss in list(Model.LOSS_TO_METHOD.keys())
        in_shape = self.layers[0].in_shape
        for i in range(1, len(self.layers)):
            self.layers[i].set_input_shape(in_shape)
            in_shape = self.layers[i].get_out_shape()
            self.layers[i].init_params()
        self.loss = Model.LOSS_TO_METHOD[loss]
        self.lr = lr
        self.compiled = True

    def summary(self):
        assert self.compiled
        base_str = '|{:10s}|{:^20s}|{:^20s}|{:^20s}|'
        print('\n' + base_str.format('Layer Type',
                                     'in shape',
                                     'out shape',
                                     'Params'))
        print('--' * 75)
        print(base_str.format(self.layers[0].LAYER_TYPE,
                              str(self.layers[0].in_shape),
                              str(self.layers[0].out_shape),
                              str(self.layers[0].nparams)))
        params = []
        for i in range(1, len(self.layers)):
            params.append(self.layers[i].get_nparams())
            print(base_str.format(self.layers[i].LAYER_TYPE,
                                  str(self.layers[i].in_shape),
                                  str(self.layers[i].get_out_shape()),
                                  str(params[-1])))
        print('-'* 75)
        print('Total Number of Params: ' + str(np.sum(params)))
        print('-'* 75 + '\n')

    def fit(self, X_train, Y_train, n_epochs=1, batch_size=256, shuffle=False):
        assert self.compiled

        for i in range(n_epochs):
            print('Epoch {}/{}'.format(i, n_epochs))
            gen = batches_generator(X_train, Y_train, batch_size, shuffle)
            l = []
            for X_, y in tqdm(gen, total=X_train.shape[0] // batch_size + 1):
                Y_hat = X_
                for k in range(1, len(self.layers)):
                    Y_hat = self.layers[k].forward(Y_hat)

                l.append(self.loss(y, Y_hat))
                dZ = self.layers[-1].backward(y)
                for k in range(len(self.layers)-1, 0, -1):
                    dZ = self.layers[k].backward(dZ)
                    self.layers[k].update_params(self.lr)
            print('\tloss: {}'.format(np.mean(l)))

    def predict(self, X, batch_size=256):
        Y_hat = []
        gen = batches_generator(X, batch_size=batch_size)
        for X_ in tqdm(gen, total=X.shape[0] // batch_size + 1):
            y_hat = X_
            for k in range(1, len(self.layers)):
                y_hat = self.layers[k].forward(y_hat)
            Y_hat.append(y_hat)
        return np.vstack(Y_hat)


In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv=KFold(n_splits=5, random_state=None, shuffle=False)

model = Model()
model.add_layers(Input([32, 32, 3]))
model.add_layers(Conv2D(4))
model.add_layers(Activation('relu'))
model.add_layers(Conv2D(8))
model.add_layers(Activation('relu'))
model.add_layers(Conv2D(16))
model.add_layers(Activation('relu'))
# model.add_layers(Conv2D(32))
# model.add_layers(Activation('relu'))
model.add_layers(Flatten())
model.add_layers(Dense(1))
model.add_layers(Activation('sigmoid'))
model.compile()
model.summary()


TRAIN_DIR = 'train/' #dataset folder heee...
TEST_DIR = 'test1/' 

ROWS = 32
COLS = 32
CHANNELS = 3

train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]
test_images = [TEST_DIR+i for i in os.listdir(TEST_DIR)]

def read_image(file_path):
  #print(file_path)
  img = cv2.imread(file_path, cv2.IMREAD_COLOR)
  #print(img)
  return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


def prep_data(images):
  m = len(images)
  n_x = ROWS*COLS*CHANNELS
  
  X = np.ndarray((m,ROWS,COLS,CHANNELS), dtype=np.uint8)
  y = np.zeros((m,1))
  print("X.shape is {}".format(X.shape))
  
  for i,image_file in enumerate(images) :
    image = read_image(image_file)
    X[i,:] = np.squeeze(image.reshape((ROWS, COLS, CHANNELS)))
    if 'dog' in image_file.lower() :
      y[i][0] = 1
    elif 'cat' in image_file.lower() :
      y[i][0] = 0
    else : # for test data
      y[i][0] = image_file.split('/')[-1].split('.')[0]
      
    if i%5000 == 0 :
      print("Proceed {} of {}".format(i, m))
    
  return X,y

X_train, y_train = prep_data(train_images[0:5000])
X_test, test_idx = prep_data(test_images[0:5000])

print("Train shape: {}".format(X_train.shape))
print("Test shape: {}".format(X_test.shape))


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)

print("Train shape: {}".format(X_train.shape))
print("Train label shape: {}".format(y_train.shape))
print("Validation shape: {}".format(X_val.shape))
print("Validation label shape: {}".format(y_val.shape))


print('\n' + '-'*33 + 'Training ' + '-' * 33)

scores = cross_val_score(model.fit(X_train, y_train, shuffle=True, n_epochs=20), X_train, y_train, scoring='accuracy', cv=cv, n_jobs=3)

model.fit(X_train, y_train, shuffle=True, n_epochs=20)

print('\n' + '-'*34 + 'Predict ' + '-' * 34)
Y_hat = model.predict(X_val)

# print(y_val)
# print(Y_hat)
new_y_hat=[]
new_y_val=[]
for i in Y_hat:
    k=int(i)
    new_y_hat.append(k)
for i in y_val:
    k=int(i)
    new_y_val.append(k)
new_y_val=np.asarray(new_y_val)
new_y_hat=np.asarray(new_y_hat)




print(y_val.shape)
print(Y_hat.shape)

print(new_y_val.shape)
print(new_y_hat.shape)
# print(new_y_val)
# print(new_y_hat)

precision=precision_score(new_y_val, new_y_hat)
recall=recall_score(new_y_val, new_y_hat)

f1_score = 2 * (precision * recall) / (precision + recall)
print("The precision is:",precision)
print("The recall is:",recall)
print("The f1 score is:",f1_score)
print("The Confusion Matrix is\n:")
print(confusion_matrix(new_y_val, new_y_hat))


# new_y_hat = np.argmax(new_y_hat, axis=0)
acc=accuracy_score(new_y_val, new_y_hat)
print('Calculated Accuracy:', np.mean(new_y_val == new_y_hat)*100)
print('Accuracy:', acc*100)


|Layer Type|      in shape      |     out shape      |       Params       |
------------------------------------------------------------------------------------------------------------------------------------------------------
|input     |    (32, 32, 3)     |    (32, 32, 3)     |         0          |
|conv      |    (32, 32, 3)     |    (16, 16, 4)     |        112         |
|act       |    (16, 16, 4)     |    (16, 16, 4)     |         0          |
|conv      |    (16, 16, 4)     |     (8, 8, 8)      |        296         |
|act       |     (8, 8, 8)      |     (8, 8, 8)      |         0          |
|conv      |     (8, 8, 8)      |     (4, 4, 16)     |        1168        |
|act       |     (4, 4, 16)     |     (4, 4, 16)     |         0          |
|flatten   |     (4, 4, 16)     |        256         |         0          |
|dense     |        256         |         1          |        257         |
|act       |         1          |         1          |         0          |
------------






  0%|          | 0/17 [00:00<?, ?it/s]

Train shape: (5000, 32, 32, 3)
Test shape: (5000, 32, 32, 3)
Train shape: (4250, 32, 32, 3)
Train label shape: (4250, 1)
Validation shape: (750, 32, 32, 3)
Validation label shape: (750, 1)

---------------------------------Training ---------------------------------
Epoch 0/20


/home/whirldata/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:279: RuntimeWarning: overflow encountered in exp





  6%|▌         | 1/17 [00:18<04:54, 18.43s/it]




 12%|█▏        | 2/17 [00:35<04:29, 17.96s/it]




 18%|█▊        | 3/17 [00:50<04:01, 17.23s/it]




 24%|██▎       | 4/17 [01:05<03:35, 16.56s/it]




 29%|██▉       | 5/17 [01:21<03:14, 16.18s/it]




 35%|███▌      | 6/17 [01:36<02:54, 15.89s/it]




 41%|████      | 7/17 [01:51<02:37, 15.77s/it]




 47%|████▋     | 8/17 [02:07<02:21, 15.71s/it]




 53%|█████▎    | 9/17 [02:23<02:05, 15.70s/it]




 59%|█████▉    | 10/17 [02:41<01:55, 16.46s/it]




 65%|██████▍   | 11/17 [02:57<01:38, 16.37s/it]




 71%|███████   | 12/17 [03:13<01:21, 16.21s/it]




 76%|███████▋  | 13/17 [03:29<01:04, 16.07s/it]




 82%|████████▏ | 14/17 [03:44<00:47, 15.88s/it]




 88%|████████▊ | 15/17 [04:00<00:31, 15.93s/it]




 94%|█████████▍| 16/17 [04:16<00:15, 15.95s/it]




100%|██████████| 17/17 [04:26<00:00, 14.08s/it]

	loss: 0.10315128213747853
Epoch 1/20


...

	loss: 0.10630518826059333
Epoch 19/20







  6%|▌         | 1/17 [00:11<03:02, 11.39s/it]




 12%|█▏        | 2/17 [00:24<02:59, 11.97s/it]




 18%|█▊        | 3/17 [00:36<02:44, 11.77s/it]




 24%|██▎       | 4/17 [00:46<02:29, 11.50s/it]




 29%|██▉       | 5/17 [00:58<02:18, 11.53s/it]




 35%|███▌      | 6/17 [01:09<02:05, 11.43s/it]




 41%|████      | 7/17 [01:20<01:53, 11.37s/it]




 47%|████▋     | 8/17 [01:32<01:42, 11.36s/it]




 53%|█████▎    | 9/17 [01:43<01:29, 11.22s/it]




 59%|█████▉    | 10/17 [01:54<01:19, 11.33s/it]




 65%|██████▍   | 11/17 [02:05<01:07, 11.30s/it]




 71%|███████   | 12/17 [02:17<00:56, 11.35s/it]




 76%|███████▋  | 13/17 [02:28<00:45, 11.31s/it]




 82%|████████▏ | 14/17 [02:39<00:33, 11.32s/it]




 88%|████████▊ | 15/17 [02:50<00:22, 11.20s/it]




 94%|█████████▍| 16/17 [03:02<00:11, 11.32s/it]




100%|██████████| 17/17 [03:09<00:00, 10.02s/it]




  0%|          | 0/3 [00:00<?, ?it/s]

	loss: 0.09999737601436369

----------------------------------Predict ----------------------------------







 33%|███▎      | 1/3 [00:03<00:07,  3.61s/it]




 67%|██████▋   | 2/3 [00:07<00:03,  3.55s/it]




100%|██████████| 3/3 [00:11<00:00,  3.74s/it]

(750, 1)
(750, 1)
(750,)
(750,)
The precision is: 0.4672131147540984
The recall is: 0.9771428571428571
The f1 score is: 0.6321626617375231
The Confusion Matrix is
:
[[ 10 390]
 [  8 342]]
Calculated Accuracy: 46.93333333333333
Accuracy: 46.93333333333333
